In [2]:
import tensorflow as tf
import numpy as np
sess=tf.Session()

In [12]:
data = np.load('/workspace/data/nctu_cgvlab_bballgan/Reordered_Data/50Real.npy')[:,:,:11,:2]
data = np.reshape(data,[data.shape[0],data.shape[1],data.shape[2]*data.shape[3]])
data_feat = np.load('/workspace/data/nctu_cgvlab_bballgan/Reordered_Data/RealCond.npy')
input_ = np.concatenate([data,data_feat], axis=-1)
print(input_.shape)
fake = tf.placeholder(tf.float32, shape=[None, 50, 28])

(14032, 50, 28)


In [37]:
ball_pos = fake[:, :, 0:2]
ball_status = fake[:, :, 22:]
ballpass_frames = tf.equal(tf.reduce_sum(ball_status, axis=-1), 0)[:, 1:-1]
vel_1 = ball_pos[:, 1:-1] - ball_pos[:, 0:-2]
vel_2 = ball_pos[:, 2:] - ball_pos[:, 1:-1]
dot_p = vel_1[:,:,0]*vel_2[:,:,0] + vel_1[:,:,1]*vel_2[:,:,1]
vel_1_norm = tf.math.sqrt(vel_1[:,:,0]**2+vel_1[:,:,1]**2+1e-10)
vel_2_norm = tf.math.sqrt(vel_2[:,:,0]**2+vel_2[:,:,1]**2+1e-10)
v = dot_p/(vel_1_norm*vel_2_norm)
clip = tf.clip_by_value(v, -1.0+1e-5, 1.0-1e-5)
theta = tf.math.acos(clip)
pass_theta = tf.cast(ballpass_frames, tf.float32)*theta
frames = tf.cast(tf.math.count_nonzero(ballpass_frames),tf.float32)
result = tf.reduce_sum(pass_theta)/frames
return_ = tf.where(tf.equal(frames, 0.0), 0.0, result)

In [47]:
a=103
feed = {fake:input_[a:a+1]}
print(sess.run(result,feed_dict=feed))

nan
